## Code search using embeddings

This notebook shows how Ada embeddings can be used to implement semantic code search. For this demonstration, we use our own [openai-python code repository](https://github.com/openai/openai-python). We implement a simple version of file parsing and extracting of functions from python files, which can be embedded, indexed, and queried.

In [9]:
import sys
import os

basename = os.path.basename(os.path.abspath('.'))

examplePath = os.path.abspath('./')
utilPath = os.path.abspath('./utils/')
print(examplePath, utilPath)

if not examplePath in sys.path:
	sys.path.append(examplePath)

if not utilPath in sys.path:
	sys.path.append(utilPath)

print(sys.path)


/workspaces/openai-cookbook/examples /workspaces/openai-cookbook/examples/utils
['/workspaces/openai-cookbook/examples', '/home/codespace/.python/current/lib/python310.zip', '/home/codespace/.python/current/lib/python3.10', '/home/codespace/.python/current/lib/python3.10/lib-dynload', '', '/home/codespace/.local/lib/python3.10/site-packages', '/home/codespace/.python/current/lib/python3.10/site-packages', '/workspaces/openai-cookbook/examples/utils']


### Helper Functions

We first setup some simple parsing functions that allow us to extract important information from our codebase.

In [10]:
import pandas as pd
from pathlib import Path

DEF_PREFIXES = ['def ', 'async def ']
NEWLINE = '\n'

def get_function_name(code):
    """
    Extract function name from a line beginning with 'def' or 'async def'.
    """
    for prefix in DEF_PREFIXES:
        if code.startswith(prefix):
            return code[len(prefix): code.index('(')]


def get_until_no_space(all_lines, i):
    """
    Get all lines until a line outside the function definition is found.
    """
    ret = [all_lines[i]]
    for j in range(i + 1, len(all_lines)):
        if len(all_lines[j]) == 0 or all_lines[j][0] in [' ', '\t', ')']:
            ret.append(all_lines[j])
        else:
            break
    return NEWLINE.join(ret)


def get_functions(filepath):
    """
    Get all functions in a Python file.
    """
    with open(filepath, 'r') as file:
        all_lines = file.read().replace('\r', NEWLINE).split(NEWLINE)
        for i, l in enumerate(all_lines):
            for prefix in DEF_PREFIXES:
                if l.startswith(prefix):
                    code = get_until_no_space(all_lines, i)
                    function_name = get_function_name(code)
                    yield {
                        'code': code,
                        'function_name': function_name,
                        'filepath': filepath,
                    }
                    break


def extract_functions_from_repo(code_root):
    """
    Extract all .py functions from the repository.
    """
    code_files = list(code_root.glob('**/*.py'))

    num_files = len(code_files)
    print(f'Total number of .py files: {num_files}')

    if num_files == 0:
        print('Verify openai-python repo exists and code_root is set correctly.')
        return None

    all_funcs = [
        func
        for code_file in code_files
        for func in get_functions(str(code_file))
    ]

    num_funcs = len(all_funcs)
    print(f'Total number of functions extracted: {num_funcs}')

    return all_funcs

# Data Loading

We'll first load the openai-python folder and extract the needed information using the functions we defined above.

In [11]:
code_root = Path(os.path.abspath('../openai-python-examples'))

print(code_root)

# Extract all functions from the repository
all_funcs = extract_functions_from_repo(code_root)

/workspaces/openai-cookbook/openai-python
Total number of .py files: 239
Total number of functions extracted: 180


Now that we have our content, we can pass the data to the `text-embedding-3-small` model and get back our vector embeddings.

In [12]:
from utils.embeddings_utils import get_embedding
df = pd.DataFrame(all_funcs)
# todo token test
df = df.head(5)
df['code_embedding'] = df['code'].apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df['filepath'] = df['filepath'].map(lambda x: Path(x).relative_to(code_root))
df.to_csv("data/code_search_openai-python.csv", index=False)
df.head()

get_embedding:  Usage(prompt_tokens=66, total_tokens=66)
get_embedding:  Usage(prompt_tokens=37, total_tokens=37)
get_embedding:  Usage(prompt_tokens=58, total_tokens=58)
get_embedding:  Usage(prompt_tokens=37, total_tokens=37)
get_embedding:  Usage(prompt_tokens=43, total_tokens=43)


,code,function_name,filepath,code_embedding
0,"def parse_obj(model: type[_ModelT], value: obj...",parse_obj,src/openai/_compat.py,"[0.0069079105742275715, 0.07924877852201462, -..."
1,def field_is_required(field: FieldInfo) -> boo...,field_is_required,src/openai/_compat.py,"[-0.003936341032385826, 0.0646929144859314, 0...."
2,def field_get_default(field: FieldInfo) -> Any...,field_get_default,src/openai/_compat.py,"[-0.0006164583610370755, -0.001203915220685303..."
3,def field_outer_type(field: FieldInfo) -> Any:...,field_outer_type,src/openai/_compat.py,"[-0.01046344731003046, 0.03492168337106705, 0...."
4,def get_model_config(model: type[pydantic.Base...,get_model_config,src/openai/_compat.py,"[0.00933151226490736, 0.015094761736690998, 0...."


### Testing

Let's test our endpoint with some simple queries. If you're familiar with the `openai-python` repository, you'll see that we're able to easily find functions we're looking for only a simple English description.

We define a search_functions method that takes our data that contains our embeddings, a query string, and some other configuration options. The process of searching our database works like such:

1. We first embed our query string (code_query) with `text-embedding-3-small`. The reasoning here is that a query string like 'a function that reverses a string' and a function like 'def reverse(string): return string[::-1]' will be very similar when embedded.
2. We then calculate the cosine similarity between our query string embedding and all data points in our database. This gives a distance between each point and our query.
3. We finally sort all of our data points by their distance to our query string and return the number of results requested in the function parameters. 

In [13]:
from utils.embeddings_utils import cosine_similarity

def search_functions(df, code_query, n=3, pprint=True, n_lines=7):
    embedding = get_embedding(code_query, model='text-embedding-3-small')
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = df.sort_values('similarities', ascending=False).head(n)
    if pprint:
        for r in res.iterrows():
            print(f"{r[1].filepath}:{r[1].function_name}  score={round(r[1].similarities, 3)}")
            print("\n".join(r[1].code.split("\n")[:n_lines]))
            print('-' * 70)

    return res

In [14]:
# res = search_functions(df, 'fine-tuning input data validation logic', n=3)
res = search_functions(df, 'get field default', n=3)

get_embedding:  Usage(prompt_tokens=3, total_tokens=3)
src/openai/_compat.py:field_get_default  score=0.605
def field_get_default(field: FieldInfo) -> Any:
    value = field.get_default()
    if PYDANTIC_V2:
        from pydantic_core import PydanticUndefined

        if value == PydanticUndefined:
            return None
----------------------------------------------------------------------
src/openai/_compat.py:field_is_required  score=0.379
def field_is_required(field: FieldInfo) -> bool:
    if PYDANTIC_V2:
        return field.is_required()
    return field.required  # type: ignore


----------------------------------------------------------------------
src/openai/_compat.py:field_outer_type  score=0.372
def field_outer_type(field: FieldInfo) -> Any:
    if PYDANTIC_V2:
        return field.annotation
    return field.outer_type_  # type: ignore


----------------------------------------------------------------------


In [15]:
res = search_functions(df, 'find common suffix', n=2, n_lines=10)

get_embedding:  Usage(prompt_tokens=3, total_tokens=3)
src/openai/_compat.py:field_outer_type  score=0.149
def field_outer_type(field: FieldInfo) -> Any:
    if PYDANTIC_V2:
        return field.annotation
    return field.outer_type_  # type: ignore


----------------------------------------------------------------------
src/openai/_compat.py:field_get_default  score=0.097
def field_get_default(field: FieldInfo) -> Any:
    value = field.get_default()
    if PYDANTIC_V2:
        from pydantic_core import PydanticUndefined

        if value == PydanticUndefined:
            return None
        return value
    return value

----------------------------------------------------------------------


In [16]:
res = search_functions(df, 'Command line interface for fine-tuning', n=1, n_lines=20)

get_embedding:  Usage(prompt_tokens=7, total_tokens=7)
src/openai/_compat.py:get_model_config  score=0.213
def get_model_config(model: type[pydantic.BaseModel]) -> Any:
    if PYDANTIC_V2:
        return model.model_config
    return model.__config__  # type: ignore


----------------------------------------------------------------------
